## ¿Qué es un modelo Seq2Seq?

Seq2Seq (Sequence to Sequence) es un método de traducción automática basado en una arquitectura codificador-decodificador (encoder-decoder). Mapea una secuencia de entrada a una secuencia de salida. La arquitectura consiste en dos redes neuronales recurrentes (RNNs): el codificador (encoder) y el decodificador (decoder). El codificador procesa la secuencia de entrada y genera un estado oculto (hidden state), que captura la información de la entrada. Este estado oculto se pasa al decodificador, que utiliza esta información para generar la secuencia de salida, prediciendo un token a la vez. A menudo, se utiliza un mecanismo de atención (attention) para permitir que el decodificador se centre en diferentes partes de la secuencia de entrada en cada paso de tiempo, mejorando así la precisión de las predicciones. Se usan tokens especiales como SOS (start of sequence) y EOS (end of sequence) para indicar el inicio y el final de las secuencias.

**Componentes clave:**

- Codificador (Encoder): Procesa la secuencia de entrada y genera un estado oculto que resume la información de la secuencia.
- Decodificador (Decoder): Utiliza el estado oculto del codificador para generar la secuencia de salida, prediciendo un token a la vez.
- Mecanismo de atención (Attention): Permite que el decodificador se enfoque en diferentes partes de la secuencia de entrada durante la generación de la secuencia de salida.
- Tokens especiales: Incluyen SOS (start of sequence) y EOS (end of sequence) para indicar el inicio y el final de las secuencias.

Puedes revisar ![](https://www.guru99.com/images/1/111318_0848_seq2seqSequ1.png)

El gráfico anterior muestra una arquitectura típica de un modelo Seq2Seq (Sequence to Sequence) con un codificador (encoder) y un decodificador (decoder) para tareas de traducción automática. Vamos a desglosar cada componente del gráfico y su funcionamiento:

* Embed (Embedding Layer): La capa de embedding toma las palabras de la secuencia de entrada ("He", "loved", "to", "eat", ".") y las convierte en vectores de alta dimensión. Esta representación densa y continua facilita que el modelo capture las relaciones semánticas entre las palabras.

* Encoder: El codificador (encoder) es una red neuronal recurrente (RNN) que procesa la secuencia de entrada. En cada paso de tiempo, la RNN toma una palabra (representada como un vector de embedding) y actualiza su estado oculto. La secuencia de estados ocultos generada por la RNN del encoder contiene la información codificada de la secuencia de entrada completa.

* S (estado oculto final del codificador): El último estado oculto del codificador (S) actúa como un resumen de toda la secuencia de entrada y se pasa al decodificador como el estado inicial. Este vector contiene la información necesaria para generar la secuencia de salida correspondiente.

* Decoder: El decodificador (decoder) es otra RNN que genera la secuencia de salida ("Er", "liebte", "zu", "essen", "."). En cada paso de tiempo, el decodificador toma el estado oculto anterior y el token generado previamente (o el token de inicio en el primer paso) para predecir el siguiente token en la secuencia de salida. El decodificador continúa este proceso hasta que se genera un token de fin de secuencia (no mostrado explícitamente en el gráfico).

* NULL token: El gráfico muestra un token "NULL" como entrada inicial al decodificador, que a menudo es un token de inicio especial (SOS - Start of Sequence) para indicar el comienzo de la secuencia de salida.

* Softmax layer: La capa softmax se aplica a la salida del decodificador en cada paso de tiempo para generar una distribución de probabilidad sobre el vocabulario de salida. La palabra con la mayor probabilidad se selecciona como la predicción del siguiente token en la secuencia de salida.

* Secuencia de salida: El decodificador produce la secuencia de salida palabra por palabra, generando tokens como "Er", "liebte", "zu", "essen", ".".


### 1. Cargando los datos

See utilizará un conjunto de datos desde [ Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/) de inglés a indonesio. 

In [ ]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
import os
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2.Preparación de datos

No se puedes usar el conjunto de datos directamente. Se necesita dividir las oraciones en palabras y convertirlas en vectores one-hot. Cada palabra se indexará de forma única en la clase `Lang` para hacer un diccionario. `Lang Class` almacenará cada oración y realizará una división palabra por palabra con `addSentence`. Luego se crea un diccionario indexando cada palabra desconocida.

In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

# Inicializamos la clase Lang

class Lang:
    def __init__(self):
        #inicializamos contenedores para almacenar las palabras y el correspondiente indice
        self.word2index = {}
        self.word2count = {}
        self.index2word ={0:"SOS",1: "EOS"}
        self.n_words = 2 # Cuenta SOS y EOS
    
    #Dividimos una oración en palabras y lo agregamos al contenedor
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    # Si la palabra no está en el contenedor, la palabra debe ser agregada
    # sino se actualiza el contador de palabras
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] +=1
    

La clase `Lang` es una clase que nos ayudará a hacer un diccionario. Para cada lenguaje, cada oración se dividirá en palabras y luego se agregará al contenedor. Cada contenedor almacenará las palabras en el índice apropiado, contará la palabra y agregará el índice de la palabra para que podamos usarlo para encontrar el índice de una palabra o encontrar una palabra de su índice.

Por cada oración que se tenga,

* Se normalizará a minúsculas,
* Se eliminará todo lo que no sea de carácter
* Se convertirá a ASCII desde Unicode
* Se dividirá las oraciones.

In [ ]:
# Normaliza cada oración

def normalize_sentence(df, lang):
    sentence = df[lang].str.lower()
    sentence = sentence.str.replace('[^A-Za-z\s]+', '', regex=True)
    sentence = sentence.str.normalize('NFD')
    sentence = sentence.str.encode('ascii', errors ='ignore').str.decode('utf-8')
    
    return sentence

def read_sentence(df, lang1, lang2):
    sentence1 = normalize_sentence(df, lang1)
    sentence2 = normalize_sentence(df, lang2)
    return sentence1, sentence2

def read_file(loc, lang1, lang2):
    df = pd.read_csv(loc, delimiter='\t', header=None, names=[lang1, lang2])
    return df

def process_data(lang1, lang2):
    df = read_file('%s-%s.txt' %(lang1, lang2), lang1, lang2)
    print("Leer %s oraciones pares" %len(df))
    sentence1, sentence2 = read_sentence(df, lang1, lang2)
    
    source = Lang()
    target = Lang()
    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]]
            source.addSentence(sentence1[i])
            target.addSentence(sentence2[i])
            pairs.append(full)
            
    return source, target, pairs
            


Otra función útil que utilizará es la conversión de pares en Tensor. Esto es muy importante por que la red solo lee datos de tipo tensorial. También es importante porque esta es la parte que en cada extremo de la oración habrá un token para indicar a la red que la entrada ha finalizado. Para cada palabra en la oración, obtendrá el índice de la palabra apropiada en el diccionario y agregará un token al final de la oración.

In [ ]:
def indexesFromSentences(lang, sentences):
    return [lang.word2index[word] for word in sentences.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentences(lang,sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device =device).view(-1,1)

def tensorsFromPair(input_lang, output_lang, pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


- Conversión a Índices: La función indexesFromSentences(lang, sentence) convierte una oración en una lista de índices basados en el diccionario (word2index) del objeto Lang. Cada palabra en la oración se convierte a su índice correspondiente.
- Conversión a Tensor: La función tensorFromSentence(lang, sentence) toma la lista de índices generada por indexesFromSentences y añade un token de fin de secuencia (EOS_token) al final de la lista. Luego, convierte esta lista en un tensor de PyTorch, asegurándose de que cada índice esté en una nueva fila, formando un vector columna.
- Pares de Tensores: La función tensorsFromPair(input_lang, output_lang, pair) toma un par de oraciones (oración de entrada y oración de salida) y las convierte en un par de tensores utilizando tensorFromSentence para cada oración. Devuelve una tupla de tensores listos para ser procesados por la red.

(EOS): Este token es crucial para indicar a la red cuándo termina una oración. Sin este token, la red no tendría una señal clara de cuándo dejar de procesar la entrada o la salida, lo que podría llevar a errores en la predicción y la generación de secuencias.


### Modelo Seq2seq 

![](https://www.guru99.com/images/1/111318_0848_seq2seqSequ2.png)

1 .Codificador (encoder):

- Las primeras tres cajas representan el codificador.
- Cada caja simboliza una celda de una red neuronal recurrente (RNN), como LSTM o GRU.
- Las letras A, B y C representan los tokens de la secuencia de entrada.

2 . Estado de contexto (context state):

- La caja al final de la secuencia del codificador (después de C) almacena el estado oculto final del codificador.
- Este estado contiene la información resumida de toda la secuencia de entrada y se pasa al decodificador.

3 . Decodificador (decoder):

- Las últimas cinco cajas representan el decodificador.
- Similar al codificador, cada caja es una celda de RNN.
- El decodificador toma el estado oculto final del codificador como su estado inicial.

4 .Tokens de entrada y salida:

- <go>: Token especial que indica el comienzo de la secuencia de salida.
- W, X, Y, Z: Tokens generados por el decodificador durante el proceso de decodificación.
- <eos>: Token especial que indica el fin de la secuencia de salida.

**Flujo del proceso**

1 . Codificación de la secuencia de entrada:

- Entrada A: El primer token de la secuencia de entrada (A) se pasa al codificador.
- Entrada B: El segundo token de la secuencia de entrada (B) se pasa a la siguiente celda del codificador.
- Entrada C: El tercer token de la secuencia de entrada (C) se pasa a la última celda del codificador.
- Estado de contexto: El estado oculto final del codificador se obtiene después de procesar C. Este estado contiene la información acumulada de A, B y C.

2 . Inicio de la decodificación:

- Token <go>: El decodificador toma el estado oculto final del codificador y comienza el proceso de generación de la secuencia de salida. El token <go> se usa para iniciar la decodificación.

3 . Generación de la secuencia de salida:

- Salida W: El decodificador genera el primer token de la secuencia de salida (W) basado en el estado oculto inicial.
- Salida X: El decodificador toma W y su estado oculto actualizado para generar el siguiente token (X).
- Salida Y: El decodificador toma X y su estado oculto actualizado para generar el siguiente token (Y).
- Salida Z: El decodificador toma Y y su estado oculto actualizado para generar el siguiente token (Z).
Token <eos>: Finalmente, el decodificador genera el token <eos> para indicar el fin de la secuencia de salida.

**Detalles adicionales**

- RNN Celdas: Cada caja en el codificador y el decodificador puede ser una celda LSTM o GRU que procesa secuencias de manera recurrente.
- Estado Oculto: El estado oculto se actualiza en cada paso de tiempo y contiene la información relevante de la secuencia hasta ese punto.
- Atención: Aunque no se muestra en esta figura, en arquitecturas más avanzadas, se puede agregar un mecanismo de atención para permitir que el decodificador enfoque diferentes partes de la secuencia de entrada en cada paso de tiempo.


![](https://www.guru99.com/images/1/111318_0848_seq2seqSequ3.png)

El modelo Seq2Seq es un tipo de modelo de aprendizaje profundo que utiliza un codificador (encoder) y un decodificador (decoder) para transformar una secuencia de entrada en una secuencia de salida. La figura muestra el flujo básico del modelo:

- El codificador toma la secuencia de entrada palabra por palabra y la convierte en una representación interna (estado oculto). Cada palabra se convierte en un índice correspondiente en el vocabulario.
- Este estado oculto resume la información de toda la secuencia de entrada y se pasa al decodificador.
- El estado oculto final del codificador contiene la información codificada de la secuencia de entrada. Este estado se transfiere al decodificador y actúa como el estado inicial del decodificador.
- El decodificador toma el estado oculto del codificador y genera la secuencia de salida, decodificando la información recibida.
- En cada paso, el decodificador puede utilizar el token generado en el paso anterior como entrada para el siguiente paso, un proceso conocido como "teacher forcing" si se usa la salida correcta en lugar de la generada.
- A cada secuencia se le asigna un token especial al final para marcar el fin de la secuencia (token <eos>).
- Un token especial al final de la secuencia de entrada indica el fin de la entrada, y un token especial al final de la salida. 

![](https://www.guru99.com/images/1/111318_0848_seq2seqSequ4.png)

El codificador procesa la oración de entrada palabra por palabra en secuencia. Al final de la secuencia de entrada, el codificador produce un estado oculto que resume toda la información de la oración. El codificador consiste en una capa de embedding y capas de GRU. La capa de embedding es una tabla de búsqueda que almacena los embeddings de las palabras de entrada en un diccionario de palabras de tamaño fijo. Estos embeddings se pasan a una capa de GRU. La capa de GRU es una unidad recurrente que calcula las representaciones ocultas de las palabras en secuencia, actualizando su estado oculto a medida que procesa cada palabra.

El flujo es:

1. Entrada secuencial: La secuencia de entrada, "how are you ?", se procesa palabra por palabra.

2. Capa embedding:
    - Función: Convierte cada palabra en un vector denso de dimensiones fijas utilizando una tabla de búsqueda (embedding matrix). Esto transforma las palabras en índices (word2id) que luego se convierten en embeddings.
    - Proceso: Cada palabra se convierte a su índice correspondiente y luego a su vector de embedding.
3. Capa GRU (Gated Recurrent Unit):
    - Función: Procesa la secuencia de embeddings. Es un tipo de RNN (Red Neuronal Recurrente) que maneja dependencias a largo plazo en la secuencia.
    - Proceso: Toma cada embedding y calcula el estado oculto. La GRU actualiza sus estados utilizando las puertas de actualización y reinicio para controlar el flujo de información.

4. Estado final: El estado oculto final del codificador resume la información de toda la secuencia de entrada y se pasa al decodificador.

Token Especial: Token de Inicio (<GO>): Este token especial indica el comienzo de la secuencia de salida y es la primera entrada para el decodificador.



El decodificador toma el estado oculto final del codificador y genera la secuencia de salida palabra por palabra. Intentará predecir la próxima palabra de salida y utilizará esta predicción como la siguiente entrada si es posible. El decodificador consiste en una capa de embedding, una capa de GRU y una capa lineal. La capa de embedding crea una tabla de búsqueda para convertir las palabras de salida en vectores de embedding. Estos embeddings se pasan a una capa de GRU, que calcula los estados ocultos para predecir las siguientes palabras en la secuencia. Finalmente, una capa lineal toma los estados ocultos y los transforma en probabilidades sobre el vocabulario de salida, utilizando una función de activación como softmax para determinar la palabra más probable.


El flujo es:


5. Capa Embedding:
    - Función: Similar a la capa de embedding del codificador, convierte los índices de las palabras en vectores de embedding. Esta vez, para las palabras de la secuencia de salida.
    - Proceso: Convierte el token `<GO>` y las palabras previas generadas en vectores de embedding.

6.  Capa GRU:
    - Función: Procesa las secuencias de embeddings de salida, utilizando el estado oculto final del codificador como su estado inicial.
    - Proceso: Genera un nuevo estado oculto y predice el siguiente token en la secuencia de salida.

7. Capa lineal:
    - Función: Transforma el estado oculto generado por la GRU en una distribución de probabilidad sobre el vocabulario de salida.
    - Proceso: Utiliza una función de activación (como softmax) para determinar la palabra más probable como salida.



El  código define un modelo Seq2Seq utilizando PyTorch, compuesto por un codificador (Encoder), un decodificador (Decoder) y una clase Seq2Seq que los une para realizar tareas de traducción de secuencias. 

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()
        
        self.input_dim = input_dim
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers =self.num_layers)
        
        
    def forward(self, src):
        embedded = self.embedding(src).view(-1, 1, self.embbed_dim)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()
        
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden): # input es (1, batch_size)
        input = input.view(1, -1)
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)       
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()
        
      # inicionalimos el encoder y decoder
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
     
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        input_length = source.size(0) 
        batch_size = target.shape[1] 
        target_length = target.shape[0]
        vocab_size = self.decoder.output_dim
      
    #inicializamos una variable para las salidas predecidas
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

    #codificamos cada palabra en una oracion
        for i in range(input_length):
            encoder_output, encoder_hidden = self.encoder(source[i])

    # utilizar la capa oculta del codificador como el decodificador oculto
        decoder_hidden = encoder_hidden.to(device)
      
    # agregamos un token antes de la primera palabra predicha
        decoder_input = torch.tensor([SOS_token], device=device)  # SOS

    # Se usa para obtener el valor de K en una lista 
        for t in range(target_length):   
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            input = (target[t] if teacher_force else topi)
            if(teacher_force == False and input.item() == EOS_token):
                break
                
        return outputs
    

El codificador toma una secuencia de entrada (palabras) y las convierte en una representación interna (estado oculto).

Funcionamiento:

- Se configura las dimensiones de entrada, ocultas y de embedding, y el número de capas GRU.
- La capa de embedding convierte las palabras de entrada en vectores densos (embeddings).
- La capa GRU procesa los embeddings secuencialmente y produce un estado oculto que contiene la información de la secuencia de entrada.
- El método forward: Toma una secuencia de entrada, la convierte en embeddings, la procesa con la GRU y devuelve las salidas y el estado oculto final.


El decodificador toma el estado oculto del codificador y genera una secuencia de salida palabra por palabra.

Funcionamiento:

-  Se configura las dimensiones de salida, ocultas y de embedding, y el número de capas GRU.
-  La capa de embedding convierte las palabras de salida en vectores densos (embeddings).
- La capa GRU procesa los embeddings secuencialmente, utilizando el estado oculto del codificador como su estado inicial.
- La capa lineal transforma el estado oculto generado por la GRU en una distribución de probabilidad sobre el vocabulario de salida.
- El método forward toma una palabra de entrada y el estado oculto, los convierte en embeddings, los procesa con la GRU y predice la siguiente palabra en la secuencia.

La clase seq2seq coordina el funcionamiento del codificador y el decodificador para transformar una secuencia de entrada en una secuencia de salida.

Funcionamiento:

- Toma el codificador y el decodificador, y establece el dispositivo para las operaciones tensoriales.
 - El método forward procesa cada palabra de la secuencia de entrada utilizando el codificador para generar un estado oculto final.
- La inicialización del decodificador configura el decodificador para empezar con el estado oculto final del codificador y un token de inicio (SOS_token).
-  En la decodificación se genera la secuencia de salida palabra por palabra. Se utiliza "teacher forcing" con una probabilidad (teacher_forcing_ratio), lo que significa que a veces usará la palabra real de la secuencia de salida como entrada para el siguiente paso, y a veces usará la palabra predicha.

- En la salida se devuelve las predicciones de salida.

### 3. Entrenamiento del modelo

El proceso de entrenamiento se inicia al convertir cada par de oraciones en tensores utilizando los índices del vocabulario de la clase Lang. El modelo utilizará el optimizador SGD (Stochastic Gradient Descent) y la función de pérdida NLLLoss (Negative Log Likelihood Loss) para calcular las pérdidas.

In [ ]:
teacher_forcing_ratio = 0.5

def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss =0
    
    output = model(input_tensor, target_tensor)
    
    num_iter = output.size(0)
    print(num_iter)
    
    # Calculamos la perdida desde una sentencia predicha con un resultado esperado
    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot])
        
        loss.backward(retain_graph=True)
        model_optimizer.step()
        epoch_loss = loss.item()/num_iter
        
        return epoch_loss
        
def train_Model(model, source, target, pairs, num_iteration = 20000):
    model.train()
    
    optimizer = optim.SGD(model.parameters(), lr =0.01)
    criterion = nn.NLLLoss()
    total_loss_iterations = 0
        
    training_pairs = [tensorsFromPair(source, target, random.choice(pairs)) for i in range(num_iteration)]
        
    for iter in range(1, num_iteration +1):
        training_pair = training_pairs[iter -1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
            
        loss =clacModel(model, input_tensor, target_tensor, optimizer, criterion)
            
        total_loss_iterations += loss
        
        if iter % 5000 == 0:
            avarage_loss = total_loss_iterations / 5000
            total_loss_iterations = 0
            print('%d %.4f' % (iter, avarage_loss))
            
    return model
    

### 4. Prueba del modelo

El proceso de evaluación consiste en verificar el resultado del modelo. Cada par de oraciones se incorporará al modelo y generará las palabras pronosticadas. Después de eso, buscará el valor más alto en cada salida para encontrar el índice correcto y al final, comparará para ver este modelo de predicción con la oración verdadera.

In [ ]:
def evaluate(model, input_lang, output_lang, sentences, max_length = MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])
        
        decoded_words = []
        
        output = model(input_tensor, output_tensor)
        
        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)
            
            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
                
    return decoded_words

def evaluateRandomly(model, source, target, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('fuente {}'.format(pair[0]))
        print('objetivo {}'.format(pair[1]))
        output_words = evaluate(model, source, target, pair)
        output_sentence = ' '.join(output_words)
        print('predicho {}'.format(output_sentence))
            
            

### Usando BLUE

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference, candidate):
    smooth = SmoothingFunction().method4
    return sentence_bleu([reference.split()], candidate.split(), smoothing_function=smooth)

def evaluate_bleu_score(model, source, target, pairs, n=10):
    bleu_scores = []
    for i in range(n):
        pair = random.choice(pairs)
        fuente = pair[0]
        objetivo = pair[1]
        output_words = evaluate(model, source, target, pair)
        predicho = ' '.join(output_words)
        bleu_score = calculate_bleu(objetivo, predicho)
        bleu_scores.append(bleu_score)
        print(f'Fuente: {fuente}')
        print(f'Objetivo: {objetivo}')
        print(f'Predicho: {predicho}')
        print(f'Puntuacion BLUE: {bleu_score:.4f}')
        print('')
    return bleu_scores


Ahora, comencemos el entrenamiento, con un número de iteraciones de 75000, el número capas RNN de 1 y con el tamaño oculto de 512.

In [ ]:
import pandas as pd
import random
import torch

# Definimos los lenguajes
lang1 = 'eng'
lang2 = 'ind'

# Procesamos los datos
source, target, pairs = process_data(lang1, lang2)

# Seleccionamos un par de oraciones aleatorio
randomize = random.choice(pairs)
print('Oración aleatoria {}'.format(randomize))

# Imprimimos el número de palabras
input_size = source.n_words
output_size = target.n_words

print('Entrada: {} Salida: {}'.format(input_size, output_size))

# Definimos los parámetros del modelo
embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 100

# Creamos el modelo encoder-decoder
encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)

# Entrenamos el modelo
model = train_Model(model, source, target, pairs, num_iteration)

# Evaluamos aleatoriamente y almacenamos resultados
def evaluateAndStoreResults(model, source, target, pairs, n=10):
    data = {
        'Fuente': [],
        'Objetivo': [],
        'Predicho': [],
        'Longitud de la Predicción': []
    }
    
    for i in range(n):
        pair = random.choice(pairs)
        fuente = pair[0]
        objetivo = pair[1]
        
        output_words = evaluate(model, source, target, pair)
        predicho = ' '.join(output_words)
        
        data['Fuente'].append(fuente)
        data['Objetivo'].append(objetivo)
        data['Predicho'].append(predicho)
        data['Longitud de la Predicción'].append(len(output_words))
        
        # Solo imprimir las oraciones, no las longitudes
        print('Fuente: {}'.format(fuente))
        print('Objetivo: {}'.format(objetivo))
        print('Predicho: {}'.format(predicho))
        print('')
    
    return pd.DataFrame(data)

# Generamos y mostramos los resultados
results_df = evaluateAndStoreResults(model, source, target, pairs, n=10)

# Guardamos los resultados en un archivo CSV para análisis posterior
results_df.to_csv('seq2seq_results.csv', index=False)
print("Resultados guardados en 'seq2seq_results.csv'")


Los números que ves en la salida del modelo representan la longitud de las secuencias generadas durante el proceso de decodificación. Cada número corresponde a la longitud de una oración predicha en términos de tokens generados.

Los resultados de traducción incluyen la oración de entrada (fuente), la oración objetivo (objetivo) y la oración predicha (predicho). 

In [ ]:
evaluateRandomly(model, source, target, pairs)

# Calcular y mostrar las puntuaciones BLEU
bleu_scores = evaluate_bleu_score(model, source, target, pairs, n=10)
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f'Puntuacion BLEU promedio: {average_bleu:.4f}')

### Ejercicios

- Experimenta con diferentes tamaños para los embeddings y las capas ocultas de tu modelo. Por ejemplo, puedes probar con un tamaño de embedding de 512 y una capa oculta de 1024.

- Añade más capas puede ayudar al modelo a capturar características más complejas del lenguaje. Prueba con 2 o 3 capas en lugar de solo una.

- Si es posible, utiliza un dataset más grande para entrenar tu modelo. Más datos pueden ayudar a que el modelo aprenda mejor las características del lenguaje.

- Asegúrate de que el preprocesamiento de tus datos sea adecuado. Considera la posibilidad de eliminar o reducir el ruido en las oraciones, normalizar contracciones y expandir abreviaturas.

- Aumenta el número de iteraciones de entrenamiento. Es posible que tu modelo aún no haya convergido adecuadamente con solo 100 iteraciones.

- Ajusta el ratio de teacher forcing. Un valor demasiado alto puede hacer que el modelo dependa en exceso del input verdadero en lugar de aprender a predecir correctamente. Experimenta con diferentes valores.

Después de implementar las mejoras, evalúa nuevamente el modelo y compara las puntuaciones BLEU.


In [ ]:
## Tus respuestas

1 . Beam Search es una técnica de búsqueda que mantiene las k mejores secuencias en cada paso del decodificador. Esto puede mejorar la calidad de las traducciones. Implementa Beam Search.



In [ ]:
def beam_search(decoder, decoder_hidden, encoder_outputs, beam_width=3, max_length=MAX_LENGTH):
    # Inicio del token
    decoder_input = torch.tensor([[SOS_token]], device=device)

    # Iniciar con un solo paso de predicción
    sequences = [[list(), 1.0]]
    completed_sequences = []

    # Paso de beam search
    for _ in range(max_length):
        all_candidates = []
        for seq, score in sequences:
            if seq and seq[-1] == EOS_token:
                completed_sequences.append((seq, score))
                continue

            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topk = decoder_output.topk(beam_width)

            for i in range(beam_width):
                candidate = [seq + [topk[1][0][i].item()], score * -topk[0][0][i].item()]
                all_candidates.append(candidate)

        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:beam_width]

    completed_sequences.extend(sequences)
    ordered = sorted(completed_sequences, key=lambda tup: tup[1])

    return ordered[0][0]

# Modificar la clase Decoder para aceptar encoder_outputs
class DecoderWithAttention(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers, attention):
        super(DecoderWithAttention, self).__init__()
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim * 2, self.output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        input = input.view(1, -1)
        embedded = F.relu(self.embedding(input))
        gru_output, hidden = self.gru(embedded, hidden)

        attn_weights = self.attention(hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))

        gru_output = gru_output.squeeze(0)
        context = context.squeeze(1)
        output = self.softmax(self.out(torch.cat([gru_output, context], 1)))

        return output, hidden, attn_weights


 Modifica la función de evaluación para usar Beam Search

In [ ]:
def evaluate_beam_search(model, input_lang, output_lang, sentences, beam_width=3, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        input_length = input_tensor.size(0)
        encoder_outputs, encoder_hidden = model.encoder(input_tensor)

        decoder_hidden = encoder_hidden.to(device)
        beam_output = beam_search(model.decoder, decoder_hidden, encoder_outputs, beam_width, max_length)

        decoded_words = [output_lang.index2word[token] for token in beam_output if token != EOS_token]

    return decoded_words


In [ ]:
## Tu respuesta

2 . Utiliza embeddings preentrenados, como GloVe, puede mejorar el rendimiento del modelo al proporcionar representaciones de palabras más ricas. Descarga e inicializa los embeddings preentrenados.

In [ ]:
import torchtext.vocab as vocab

glove = vocab.GloVe(name='6B', dim=100)

def load_embeddings(vocab, glove):
    matrix_len = len(vocab)
    weights_matrix = np.zeros((matrix_len, 100))
    words_found = 0

    for i, word in enumerate(vocab):
        try:
            weights_matrix[i] = glove[word]
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=(100,))

    return torch.tensor(weights_matrix).float()


Modificar el modelo para usar los embeddings preentrenados

In [ ]:
class EncoderWithPretrainedEmbeddings(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers, weights_matrix):
        super(EncoderWithPretrainedEmbeddings, self).__init__()
        
        self.input_dim = input_dim
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)
        self.embedding.load_state_dict({'weight': weights_matrix})
        self.embedding.weight.requires_grad = False

        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
        
        
    def forward(self, src):
        embedded = self.embedding(src).view(-1, 1, self.embbed_dim)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

weights_matrix = load_embeddings(source.word2index, glove)

encoder = EncoderWithPretrainedEmbeddings(input_size, hidden_size, embed_size, num_layers, weights_matrix)


In [ ]:
## Tu respuesta

3 . Agrega técnicas de regularización como Dropout puede ayudar a evitar el sobreajuste y mejorar el rendimiento general del modelo. Añade Dropout en el Encoder y Decoder

In [ ]:
class EncoderWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers, dropout=0.5):
        super(EncoderWithDropout, self).__init__()
        
        self.input_dim = input_dim
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers, dropout=dropout)
        
    def forward(self, src):
        embedded = self.embedding(src).view(-1, 1, self.embbed_dim)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

class DecoderWithDropout(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers, dropout=0.5):
        super(DecoderWithDropout, self).__init__()
        
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers, dropout=dropout)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden): # input es (1, batch_size)
        input = input.view(1, -1)
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)       
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden


In [ ]:
## Tu respuesta

4 . Experimenta con diferentes optimizadores y tasas de aprendizaje y después de realizar las modificaciones, vuelve a entrenar y evaluar el modelo.



In [ ]:
## Tu respuesta